# Data analysis

In [15]:
import glob, os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import sigmaclip as clip
import numpy as np
from astropy.stats import sigma_clip

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [16]:
!ls

Analysis.ipynb           cql_runtimes_5_3.json    draco_runtimes_20_5.json
chart.vl.json            cql_runtimes_5_4.json    draco_runtimes_25_1.json
cql_runtimes_10_1.json   cql_runtimes_5_5.json    draco_runtimes_25_2.json
cql_runtimes_10_2.json   draco_runtimes_10_1.json draco_runtimes_25_3.json
cql_runtimes_10_3.json   draco_runtimes_10_2.json draco_runtimes_25_4.json
cql_runtimes_10_4.json   draco_runtimes_10_3.json draco_runtimes_25_5.json
cql_runtimes_15_1.json   draco_runtimes_10_4.json draco_runtimes_5_1.json
cql_runtimes_15_2.json   draco_runtimes_10_5.json draco_runtimes_5_2.json
cql_runtimes_15_3.json   draco_runtimes_15_1.json draco_runtimes_5_3.json
cql_runtimes_20_1.json   draco_runtimes_15_2.json draco_runtimes_5_4.json
cql_runtimes_20_2.json   draco_runtimes_15_3.json draco_runtimes_5_5.json
cql_runtimes_20_3.json   draco_runtimes_15_4.json results.json
cql_runtimes_25_1.json   draco_runtimes_15_5.json run_cql_benchmark.sh
cql_runtimes_25_2.json   draco_runtimes_20_1.j

In [17]:
draco = pd.concat(map(pd.read_json, glob.glob('draco_runtimes*.json')))
cql = pd.concat(map(pd.read_json, glob.glob('cql_runtimes*.json')))

df = pd.concat([cql, draco])
df = df.reindex()

In [18]:
df

,encodings,fields,runtime,system
0,3,20,75.221,cql
1,3,20,75.431,cql
2,3,20,75.956,cql
3,3,20,75.524,cql
4,3,20,76.670,cql
5,3,20,76.054,cql
6,3,20,75.531,cql
7,3,20,77.924,cql
8,3,20,75.924,cql
9,3,20,75.781,cql


In [19]:
gb = df.groupby(['system', 'encodings', 'fields'])
results = pd.DataFrame(columns=['system', 'encodings', 'fields', 'runtime'])

for i, (name, group) in enumerate(gb):
#     print(group)
#     group.runtime.hist()
    before = group.runtime
    after = pd.Series(sigma_clip(before)).dropna()
    print(name, len(before), len(after))
    
    results = results.append(pd.DataFrame.from_dict([{
        'system': name[0],
        'encodings': name[1],
        'fields': name[2],
        'mean': np.mean(after),
        'stdev': np.std(after),
        'min': np.min(before),
        'max': np.max(before),
        'runtime': np.median(before)
    }]))
#     plt.show()

('cql', 1, 5) 20 20
('cql', 1, 10) 20 20
('cql', 1, 15) 20 20
('cql', 1, 20) 20 19
('cql', 1, 25) 20 19
('cql', 2, 5) 20 20
('cql', 2, 10) 20 16
('cql', 2, 15) 20 20
('cql', 2, 20) 20 20
('cql', 2, 25) 20 18
('cql', 3, 5) 20 17
('cql', 3, 10) 20 20
('cql', 3, 15) 20 20
('cql', 3, 20) 20 19
('cql', 3, 25) 20 20
('cql', 4, 5) 20 20
('cql', 4, 10) 20 19
('cql', 5, 5) 20 16
('draco', 1, 5) 20 18
('draco', 1, 10) 20 19
('draco', 1, 15) 20 20
('draco', 1, 20) 20 17
('draco', 1, 25) 20 18
('draco', 2, 5) 20 19
('draco', 2, 10) 20 19
('draco', 2, 15) 20 20
('draco', 2, 20) 20 19
('draco', 2, 25) 20 20
('draco', 3, 5) 20 20
('draco', 3, 10) 20 17
('draco', 3, 15) 20 17
('draco', 3, 20) 20 17
('draco', 3, 25) 20 20
('draco', 4, 5) 20 18
('draco', 4, 10) 20 18
('draco', 4, 15) 20 19
('draco', 4, 20) 20 19
('draco', 4, 25) 20 20
('draco', 5, 5) 20 18
('draco', 5, 10) 20 20
('draco', 5, 15) 20 20
('draco', 5, 20) 20 19
('draco', 5, 25) 20 20


In [20]:
results

,encodings,fields,max,mean,min,runtime,stdev,system
0,1,5,0.006,0.004500,0.002,0.0050,1.118034e-03,cql
0,1,10,0.029,0.016400,0.007,0.0165,6.280127e-03,cql
0,1,15,0.026,0.016600,0.012,0.0170,3.382307e-03,cql
0,1,20,0.042,0.021368,0.016,0.0195,4.427314e-03,cql
0,1,25,0.042,0.023579,0.016,0.0225,4.716330e-03,cql
0,2,5,0.036,0.017000,0.009,0.0150,7.416198e-03,cql
0,2,10,0.279,0.119438,0.115,0.1190,3.587631e-03,cql
0,2,15,0.430,0.331950,0.285,0.3150,4.263388e-02,cql
0,2,20,0.681,0.583950,0.535,0.5680,4.133458e-02,cql
0,2,25,1.034,0.886333,0.839,0.8805,3.082027e-02,cql


In [21]:
results.to_json('results.json',orient='records')